In [1]:
VERSION = '99_v1_02_14'

In [2]:
model_list = [    
    #{'path':'../../13_Baseline5/exp/result/13_v1_03/oof_df.csv', 'w':0.5}, #deberta-v3-base, seed100, split00_v2_07
    #{'path':'../../13_Baseline5/exp/result/13_v1_04/oof_df.csv', 'w':1}, #deberta-v3-base, seed200, split00_v2_07
    #{'path':'../../13_Baseline5/exp/result/13_v1_07/oof_df.csv', 'w':0.5}, #deberta-base, seed100, split00_v2_07
    {'path':'../../18_BaselineSW/exp/result/18_v2_01/oof_df.csv', 'w':1}, #deberta-large, seed100, split00_v2_07
    {'path':'../../18_BaselineSW/exp/result/18_v2_03/oof_df.csv', 'w':1}, #deberta-v3-large, seed100, split00_v2_07
    {'path':'../../18_BaselineSW/exp/result/18_v2_04/oof_df.csv', 'w':0.5}, #longformer-large, seed100, split00_v2_07
    {'path':'../../18_BaselineSW/exp/result/18_v2_07/oof_df.csv', 'w':1}, #deberta-large, seed200, split00_v2_07
    {'path':'../../18_BaselineSW/exp/result/18_v1_04/oof_df.csv', 'w':1}, #deberta-v3-large, seed200, split00_v2_07
]

In [3]:
import pandas as pd

w_list = []
for i,model in enumerate(model_list):
    w = model['w']
    w_list.append(w)

stack_df = None
for i,model in enumerate(model_list):
    pred_df = pd.read_csv(model['path'])
    w = w_list[i]
    pred_df['pred_ineffective'] = pred_df['pred_ineffective'] * w / sum(w_list)
    pred_df['pred_adequate'] = pred_df['pred_adequate'] * w / sum(w_list)
    pred_df['pred_effective'] = pred_df['pred_effective'] * w / sum(w_list)
    pred_df = pred_df.rename(columns={
        'pred_ineffective':f'Ineffective_{i}', 
        'pred_adequate':f'Adequate_{i}', 
        'pred_effective':f'Effective_{i}'
    })
    if stack_df is None:
        stack_df = pred_df.copy()
    else:
        stack_df = stack_df.merge(pred_df[['discourse_id',f'Ineffective_{i}',f'Adequate_{i}',f'Effective_{i}']],
                                  on='discourse_id', how='left')
    

train_df = pd.read_csv('../../input/feedback-prize-effectiveness/train.csv')
oof_df = train_df[['discourse_id']].merge(stack_df, on='discourse_id', how='left')

In [4]:
for col_name in ['Ineffective','Adequate','Effective']:
    cols = [f'{col_name}_{i}' for i in range(len(model_list))]
    oof_df[col_name] = oof_df[cols].sum(axis=1)

In [5]:
from sklearn.metrics import log_loss

score = log_loss(oof_df['label'].values, oof_df[['Ineffective','Adequate','Effective']].values)
print('oof={:.4f}'.format(score))

oof=0.5904


In [6]:
import os
os.makedirs('result', exist_ok=True)

oof_df.to_csv(f'result/stack_{VERSION}.csv', index=False)

In [7]:
oof_df.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,Ineffective_0,Adequate_0,Effective_0,label,loss,...,Effective_2,Ineffective_3,Adequate_3,Effective_3,Ineffective_4,Adequate_4,Effective_4,Ineffective,Adequate,Effective
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,0.002900,0.203973,0.015350,1,0.085691,...,0.020100,0.006687,0.209799,0.005736,0.004861,0.212263,0.005099,0.022401,0.921786,0.055813
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,0.012444,0.203638,0.006140,1,0.087332,...,0.004512,0.014994,0.202803,0.004426,0.049650,0.170513,0.002059,0.096651,0.880388,0.022961
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,0.031269,0.188478,0.002476,1,0.164698,...,0.004119,0.060817,0.159702,0.001704,0.064136,0.156996,0.001090,0.220628,0.767838,0.011533
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,0.074311,0.145993,0.001918,1,0.420119,...,0.002815,0.057894,0.161296,0.003032,0.036142,0.184023,0.002057,0.249729,0.737834,0.012437
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,0.055053,0.166231,0.000937,1,0.290297,...,0.003135,0.087955,0.132984,0.001283,0.049520,0.171717,0.000985,0.287410,0.704892,0.007699
